In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline

import seaborn as sns
sns.set(style="white",color_codes=True)
sns.set(font_scale=1.5)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# Read runways data from runways.xlsx.
link = 'https://docs.google.com/spreadsheets/d/1wJmRDIaU8sCnOhJYJ3iXavWrwminxnoz/edit?usp=drive_link'

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('runways.xlsx')

runways = pd.read_excel('runways.xlsx')

# Read Airlines data from Airlines.xlsx.
link = 'https://docs.google.com/spreadsheets/d/15UqVa1M6jvIMcIVB2VytLKo3cY8qkB5L/edit?usp=drive_link'

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Airlines.xlsx')

airlines = pd.read_excel('Airlines.xlsx')

# Read airports data from airports.xlsx.
link = 'https://docs.google.com/spreadsheets/d/1QrgxQDPDcgoW0zxTCVay49X5ixPaOn3-/edit?usp=drive_link'

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('airports.xlsx')

airports = pd.read_excel('airports.xlsx')

# Read Data Dictionary data from Data Dictionary.xlsx.
link = 'https://docs.google.com/spreadsheets/d/1qOdR5OySMsBkPaFA_QXOLJW2KN2NE0vr/edit?usp=drive_link'

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Data Dictionary.xlsx')

dataDic = pd.read_excel('Data Dictionary.xlsx')


In [4]:
airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [5]:
airlines.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0


In [6]:
runways.head()

,id,airport_ref,airport_ident,length_ft,width_ft,surface,lighted,closed,le_ident,le_latitude_deg,le_longitude_deg,le_elevation_ft,le_heading_degT,le_displaced_threshold_ft,he_ident,he_latitude_deg,he_longitude_deg,he_elevation_ft,he_heading_degT,he_displaced_threshold_ft
0,269408,6523,00A,80.0,80.0,ASPH-G,1,0,H1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,255155,6524,00AK,2500.0,70.0,GRVL,0,0,N,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN
2,254165,6525,00AL,2300.0,200.0,TURF,0,0,1,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN
3,270932,6526,00AR,40.0,40.0,GRASS,0,0,H1,NaN,NaN,NaN,NaN,NaN,H1,NaN,NaN,NaN,NaN,NaN
4,322128,322127,00AS,1450.0,60.0,Turf,0,0,1,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN


In [7]:
dataDic.head(5)

,Column,Description,Unnamed: 2
0,Airline,Different types of commercial airlines,NaN
1,Flight,Types of Aircraft,NaN
2,AirportFrom,Source Airport,NaN
3,AirportTo,Destination Airport,NaN
4,DayOfWeek,Tells you about the day of week,NaN


In [20]:
# read airplane operator details
url = 'https://en.wikipedia.org/wiki/List_of_airlines_of_the_United_States'

page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

airoperator = pd.DataFrame(columns = ['Airline','Founded'])
for table in soup.find_all('table',class_='wikitable sortable'):
    for row in table.tbody.find_all('tr'):
        columns = row.find_all('td')

        if(columns != []):
            airline = columns[0].text.strip()
            founded = columns[6].text.strip()
            d = pd.DataFrame([[airline,founded]],columns =  airoperator.columns)
            #s = pd.DataFrame({'Airline':airline,'Founded':founded},columns = airoperator.columns)
            airoperator = pd.concat([airoperator, d],ignore_index=True,axis = 0)

airoperator.head()

,Airline,Founded
0,Alaska Airlines,1932
1,Allegiant Air,1997
2,American Airlines,1926
3,Avelo Airlines,1987
4,Breeze Airways,2018


In [22]:
# Read airport trafix from wikipedia.
url = 'https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States'
r = 0
airports_traffic = pd.DataFrame(columns = ['Airports','IATA','Cities','Metro_Area','State','2022','2021','2020','2019','2018','2017','2016','2015','2014','2013'])
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
for table in soup.find_all('table',class_='wikitable sortable'):
    if r < 2:
        r = r + 1
        for row in table.tbody.find_all('tr'):
            columns = row.find_all('td')

            if(columns != []):
                airports = columns[1].text.strip()
                iata = columns[2].text.strip()
                cities = columns[3].text.strip()
                metroarea = columns[4].text.strip()
                state = columns[5].text.strip()
                year2022 = columns[6].text.strip()
                year2021 = columns[7].text.strip()
                year2020 = columns[8].text.strip()
                year2019 = columns[9].text.strip()
                year2018 = columns[10].text.strip()
                year2017 = columns[11].text.strip()
                year2016 = columns[12].text.strip()
                year2015 = columns[13].text.strip()
                year2014 = columns[14].text.strip()
                year2013 = columns[15].text.strip()

                '''df = df.append({'Airports':airports,'IATA':iata,'Cities':cities,'Metro_Area':metroarea,
                                  'State':state,'2022':year2022,'2021':year2021,'2020':year2020,'2019':year2019,
                                  '2018':year2018,'2017':year2017,'2016':year2016,'2015':year2015,'2014':year2014,'2013':year2013},ignore_index=True)'''
                df1 =  pd.DataFrame([[airports,iata,cities,metroarea,state,year2022,year2021,year2020,year2019,year2018,year2017,year2016,year2015,year2014,year2013]],columns =  airports_traffic.columns)

                airports_traffic = pd.concat([airports_traffic, df1],ignore_index=True,axis = 0)
airports_traffic.head()


,Airports,IATA,Cities,Metro_Area,State,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013
0,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,Atlanta,GA,"45,396,001","36,676,010","20,559,866","53,505,795","51,865,797","50,251,964","50,501,858","49,340,732","46,604,273","45,308,407"
1,Dallas/Fort Worth International Airport,DFW,Dallas & Fort Worth,Dallas–Fort Worth,TX,"35,345,138","30,005,266","18,593,421","35,778,573","32,821,799","31,816,933","31,283,579","31,589,839","30,804,567","29,038,128"
2,Denver International Airport,DEN,Denver,Denver,CO,"33,773,832","28,645,527","16,243,216","33,592,945","31,362,941","29,809,097","28,267,394","26,280,043","26,000,591","25,496,885"
3,O'Hare International Airport,ORD,Chicago,Chicago,IL,"33,120,474","26,350,976","14,606,034","40,871,223","39,873,927","38,593,028","37,589,899","36,305,668","33,843,426","32,317,835"
4,Los Angeles International Airport,LAX,Los Angeles,Greater Los Angeles,CA,"32,326,616","23,663,410","14,055,777","42,939,104","42,624,050","41,232,432","39,636,042","36,351,272","34,314,197","32,425,892"
